In [1]:
import requests
import json

url = "https://apis.openapi.sk.com/puzzle/congestion-car/stat/stations/211?dow=FRI&hh=18"

headers = {
    "accept": "application/json",
    "appKey": "pa9QvuuFu82jHWycxt21dan2IW3gmf55419lWWDt"
}

response = requests.get(url, headers=headers)

# 들여쓰기가 된 채로 출력
parsed_data = json.loads(response.text)
print(json.dumps(parsed_data, indent=4,ensure_ascii=False))

{
    "status": {
        "code": "00",
        "message": "success",
        "totalCount": 1
    },
    "contents": {
        "subwayLine": "2호선",
        "stationName": "성수역",
        "stationCode": "211",
        "stat": [
            {
                "startStationCode": "202",
                "startStationName": "을지로입구역",
                "endStationCode": "211",
                "endStationName": "성수역",
                "prevStationCode": "212",
                "prevStationName": "건대입구역",
                "updnLine": 0,
                "directAt": 0,
                "data": [
                    {
                        "dow": "FRI",
                        "hh": "18",
                        "mm": "00",
                        "congestionCar": [
                            0,
                            0,
                            0,
                            0,
                            0,
                            0,
                            0,
                       

In [22]:
type(parsed_data)

dict

In [2]:
del parsed_data['status']
parsed_data.keys()

dict_keys(['contents'])

In [3]:
parsed_data['contents']

{'subwayLine': '2호선',
 'stationName': '성수역',
 'stationCode': '211',
 'stat': [{'startStationCode': '202',
   'startStationName': '을지로입구역',
   'endStationCode': '211',
   'endStationName': '성수역',
   'prevStationCode': '212',
   'prevStationName': '건대입구역',
   'updnLine': 0,
   'directAt': 0,
   'data': [{'dow': 'FRI',
     'hh': '18',
     'mm': '00',
     'congestionCar': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
    {'dow': 'FRI',
     'hh': '18',
     'mm': '10',
     'congestionCar': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
    {'dow': 'FRI',
     'hh': '18',
     'mm': '20',
     'congestionCar': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
    {'dow': 'FRI',
     'hh': '18',
     'mm': '30',
     'congestionCar': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
    {'dow': 'FRI',
     'hh': '18',
     'mm': '40',
     'congestionCar': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
    {'dow': 'FRI',
     'hh': '18',
     'mm': '50',
     'congestionCar': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}]},
  {'startStationCode': '202',
   'startStationName'

In [18]:
counting_row = parsed_data['contents']
type(counting_row)

dict

# 여기 밑에 코드는 무시하세요. 연습장으로 쓴거에요.

In [15]:
data_count = 0

def count_data(lst):
    global data_count
    for item in lst:
        if isinstance(item, list):
            count_data(item)
        elif isinstance(item, dict):
            if 'data' in item:
                data_count += 1
                count_data(item['data'])

count_data(counting_row)
print("data 리스트가", data_count, "번 나왔습니다.")

data 리스트가 0 번 나왔습니다.


In [25]:
def count_keys_with_keyword(dictionary, keyword):
    count = 0
    for key, value in dictionary.items():
        if isinstance(value, dict):
            count += count_keys_with_keyword(value, keyword)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    count += count_keys_with_keyword(item, keyword)
                elif isinstance(item, int):
                    continue  # 딕셔너리가 아닌 경우 건너뜁니다.
                if keyword in item.keys():
                    count += 1
        if keyword in key:
            count += 1
    return count

keyword_count = count_keys_with_keyword(parsed_data['contents'], "data")
print("키워드를 포함하는 키의 개수:", keyword_count)

키워드를 포함하는 키의 개수: 44


In [ ]:
import json
import pandas as pd
from sklearn.cluster import KMeans

# JSON 데이터 읽기
data = {
    "subwayLine": "2호선",
    "stationName": "성수역",
    "stationCode": "211",
    "stat": [
        {
            "startStationCode": "202",
            "startStationName": "을지로입구역",
            "endStationCode": "211",
            "endStationName": "성수역",
            "prevStationCode": "212",
            "prevStationName": "건대입구역",
            "updnLine": 0,
            "directAt": 0,
            "data": [
                {
                    "dow": "MON",
                    "hh": "08",
                    "mm": "00",
                    "congestionCar": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                },
                ...
            ]
        }
    ]
}

# 데이터 가공 및 추출
dow_list = []
time_list = []
updn_line_list = []
congestion_list = []

for item in data['stat']:
    for train_data in item['data']:
        dow_list.append(train_data['dow'])
        time_list.append(int(train_data['hh']) * 60 + int(train_data['mm']))
        updn_line_list.append(item['updnLine'])
        congestion_list.append(train_data['congestionCar'])

# Pandas DataFrame 생성
df = pd.DataFrame({
    'DayOfWeek': dow_list,
    'Time': time_list,
    'UpdnLine': updn_line_list,
    'Congestion': congestion_list
})

# K-means 군집화 모델 학습
X = df[['Time', 'UpdnLine', 'Congestion']]
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)

# 클러스터 할당 결과 확인
df['Cluster'] = kmeans.labels_

# 클러스터 결과 출력
print(df[['DayOfWeek', 'Time', 'UpdnLine', 'Congestion', 'Cluster']])


### 통계성이 아닌 실시간 SK API

In [6]:
import requests

url = "https://apis.openapi.sk.com/puzzle/congestion-train/rltm/trains/2/2505"
headers = {
    "accept": "application/json",
    "appKey": "pa9QvuuFu82jHWycxt21dan2IW3gmf55419lWWDt"
}

response = requests.get(url, headers=headers)

# 들여쓰기가 된 채로 출력
parsed_data = json.loads(response.text)
print(json.dumps(parsed_data, indent=4,ensure_ascii=False))

# 실행 시각 6/2 23:39

{
    "success": true,
    "code": 0,
    "msg": "Request Succeeded.",
    "data": {
        "subwayLine": "2",
        "trainY": "2505",
        "congestionResult": {
            "congestionTrain": "0",
            "congestionCar": "0|0|0|0|0|0|0|0|0|0",
            "congestionType": 2
        }
    }
}


### 공공데이터가 제공하는 열차 혼잡도

In [13]:
import requests

url = "https://api.odcloud.kr/api/15071311/v1/uddi:75461a18-17a3-42fe-9322-a51148003b69?page=3&perPage=10&returnType=json&serviceKey=7R%2BUADB4OjI1NOLP1IKFLdLSoOtxqhQMeaEo%2FgF3yk%2BGCwpmsDCo3dbh51FwGmYxhCmyvWQ7yli525ggLIsivA%3D%3D"
# page는 
# perPage는 표시할 최대 인덱스

response = requests.get(url, headers=headers)

# 들여쓰기가 된 채로 출력
parsed_data = json.loads(response.text)
print(json.dumps(parsed_data, indent=4,ensure_ascii=False))

{
    "currentCount": 10,
    "data": [
        {
            "00시00분": "16.2",
            "00시30분": "4.7",
            "10시00분": "27.6",
            "10시30분": "37.1",
            "11시00분": "34.8",
            "11시30분": "34.1",
            "12시00분": "36.5",
            "12시30분": "38.9",
            "13시00분": "38.4",
            "13시30분": "37.2",
            "14시00분": "37.0",
            "14시30분": "35.2",
            "15시00분": "39.9",
            "15시30분": "35.2",
            "16시00분": "38.9",
            "16시30분": "50.0",
            "17시00분": "50.3",
            "17시30분": "52.7",
            "18시00분": "63.8",
            "18시30분": "53.6",
            "19시00분": "34.6",
            "19시30분": "25.4",
            "20시00분": "21.8",
            "20시30분": "22.3",
            "21시00분": "35.0",
            "21시30분": "33.6",
            "22시00분": "34.8",
            "22시30분": "32.3",
            "23시00분": "25.6",
            "23시30분": "21.3",
            "5시30분": "18.9",
            "6시00분": "